# 102 Queries 1 (Python)
https://weaviate.io/developers/academy/zero_to_mvp/queries_1

## Unit overview

In [1]:
from IPython.display import HTML
HTML('<iframe width="640" height="360" src="https://www.youtube.com/embed/Fk9pDfBhnTI" allowfullscreen></iframe>')

/Users/matthias/Desktop/WeaviateBasics/env/lib/python3.10/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In this unit, you will learn how to efficiently retrieve relevant objects or aggregated information from Weaviate.

You have already encountered some examples of vector searches. In this section, we will delve deeper by reviewing the various vector search methods available in Weaviate, such as `nearVector`, `nearObject`, and `nearText`.

Along with vector search methods, you will also discover filters that can be employed to accompany search operators. For instance, you will learn how to search for data objects that exclude specific criteria.

As we examine these capabilities, we will simultaneously use them as a means to gain insight into the inner workings of Weaviate.

Upon completing this unit, you will possess a thorough understanding of how to effectively query Weaviate to obtain desired results, as well as the underlying mechanisms that make it all possible.

### Prerequisites
(**Required**) A Python (3) environment with `weaviate-client` installed.
(**Required**) Complete [101A Weaviate Academy Preparation](https://weaviate.io/developers/academy/zero_to_mvp/setup).
(Recommended) Complete [Hello, Weaviate](https://weaviate.io/developers/academy/zero_to_mvp/hello_weaviate).

## Learning objectives
Each unit includes a set of Learning Goals and Learning Outcomes which form the unit's guiding principles.
- *Learning Goals* describe the unit's key topics and ideas.
- *Learning Outcomes* on the other hand describe tangible skills that the learner should be able to demonstrate.

### Here, we will cover:
Learning Goals
- How to retrieve objects and properties.
- The structure of returned responses from Weaviate.
- The difference between `nearVector`, `nearObject` and `nearText`.
- How to aggregate meta information about objects.
- How to add filters to vector searches.
- Weaviate's internal vector search process.

### By the time you are finished, you will be able to:
Learning Outcomes
- Construct "Get" queries to retrieve relevant objects and desired properties.
- Parse a response from Weaviate.
- Explain the differences between `nearVector`, `nearObject` and `nearText`.
- Construct `Aggregate` queries to retrieve aggregated properties about relevant objects.
- Add filters to queries to exclude certain results.
- Describe how `nearObject` and `nearText` queries are converted to vector searches, and what pre-filtering is.

# Get{} objects
https://weaviate.io/developers/academy/zero_to_mvp/queries_1/get

Make sure to complete the [Weaviate Academy Preparation](https://weaviate.io/developers/academy/zero_to_mvp/setup) mini-unit before starting this unit to make sure that you can run the client library and connect to the demo Weaviate instance without issues.

Below, you will see code snippets that do not include client instantiation details. Before running these snippets, make sure to instantiate the client as shown below.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
WEAVIATE_CLIENT_URL = os.getenv("WEAVIATE_CLIENT_URL")
WEAVIATE_CLIENT_KEY = os.getenv("WEAVIATE_CLIENT_KEY")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print("environment variables loaded")
#
import weaviate
client = weaviate.Client(
    url="https://edu-demo.weaviate.network",
    auth_client_secret=weaviate.AuthApiKey(api_key="learn-weaviate"), # A read-only API Key for the Weaviate instance
    additional_headers={"X-OpenAI-Api-Key": OPENAI_API_KEY} # Replace this with YOUR OpenAI API key
)
client

environment variables loaded


## Overview

In [3]:
HTML('<iframe width="640" height="360" src="https://www.youtube.com/embed/q2UuBIJVLEM" allowfullscreen></iframe>')

/Users/matthias/Desktop/WeaviateBasics/env/lib/python3.10/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## About `Get` queries
In many use cases, retrieving objects from a Weaviate instance may be the most common operation.

For example, a user may want to retrieve a list of passages most closely related to the input query, or they may wish to retrieve a list of images which are most similar to another image. It is even possible to retrieve a set of images that best match a given passage.

In Weaviate, such operations to retrieve objects are performed using the `Get` function.

### `Get` function syntax
A basic `Get` function looks as follows:
```python
response = client.query.get(
    <Class>,
    [<properties>]
).<arguments>.do()
```
- The `Class` field specifies the name of the class of objects to be retrieved.
- The `arguments` argument specifies the search criteria to be used to retrieve the objects.
- The `properties` argument specifies the properties of the objects to be retrieved, including any `_additional` properties.

Now let's try out some concrete `Get` queries.

## Standalone `Get` queries
A basic, standalone, `Get` query might look as follows:

### Example

In [4]:
import json
response = (
    client
    .query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_limit(2)
    .do()
)
print(json.dumps(response, indent=2))

{
  "data": {
    "Get": {
      "JeopardyQuestion": [
        {
          "answer": "Jonah",
          "question": "This prophet passed the time he spent inside a fish offering up prayers"
        },
        {
          "answer": "lay eggs",
          "question": "Pythons are oviparous, meaning they do this"
        }
      ]
    }
  }
}


What results do you expect? See if you can correspond each field to the syntax.

Now, try it out yourself. This query should return something like the below:

```
{
  "data": {
    "Get": {
      "JeopardyQuestion": [
        {
          "answer": "Amazon.com",
          "question": "On July 16, 1995 this company made its first sale, a science textbook"
        },
        {
          "answer": "Parfait",
          "question": "The French name of this layered ice cream dessert means \"perfect\""
        }
      ]
    }
  }
}
```

### Response object from Weaviate
As you see above, the response object from Weaviate is in JSON format, where the results are obtained via the `data` field. It is then arranged in a corresponding manner to reflect the query as well as the queried object.

In the above example, the `Get` field reflects the query function, and the `JeopardyQuestion` field reflects the queried object class, containing returned objects. Each object contains answer and `question` fields, reflecting the requested properties.

> In this case, Weaviate will return two objects due to the `.with_limit(2)` argument. Without this limit, Weaviate would return a maximum number according to its configuration.<br><br>IS THIS QUERY USEFUL?<br>
As this does not apply any user-specific criteria to the search, the specific results will likely not be very useful.<br>However, this may be a viable method for "sanity" checks, such as checking that you can connect to a Weaviate instance, or that at least some objects have been imported successfully.

### `Class` and `properties`
In the above example, we specify a `Class` of `JeopardyQuestion` and `properties` of `questions` and `answer`.

That is possible because those follow the structure of our data in Weaviate. To see the available data classes and properties, you can take a look at the Weaviate schema, as shown below:

You can fetch the schema like this - try it out!

`client.schema.get()`

```
{
  "classes": [
    {
      "class": "JeopardyQuestion",
      "properties": [
        {
          "dataType": ["text"],
          "name": "question",
          ...  // Truncated
        },       
        {
          "dataType": ["text"],
          "name": "answer",
          ...  // Truncated
        },  
        {
          "dataType": ["int"],
          "name": "points"
          ...  // Truncated
        },        
        ...  // Truncated
      ],
      ...  // Truncated
    }
  ]
}
```

The `Class` and `properties` fields must correspond to collections of objects that have been defined in the Weaviate schema.

The `Class` must be the name of a data object collection, and the `properties` a list of properties to be retrieved.

The schema contains the `JeopardyQuestion` class, with properties: `question`, `answer` and `points`.

So, a query retrieving objects from the `Question` class could specify any of its properties such as `question`, `answer`, and `points`.

> EXERCISE<br>Try out the above query again, with these changes.<br>- Can you get the points property as well?<br>- What happens if you don't specify any properties?

> CAN I SEARCH MULTIPLE CLASSES AT ONCE?<br>No. You can only search one class at a time.<br><br>This is because each class constitutes a single `vector space`. If you want to search multiple collections of objects, you will have to perform multiple searches, or consider putting them into one class and using a filter to distinguish between them as required.<br><br>We will consider this topic in more detail in a later unit, including what it means for each class to constitute a distinct `vector space`, and how to think about building a schema in Weaviate.

## `Get` with `additional` properties
You can retrieve additional properties that are not defined in the schema. These properties may be inherent to the object, or relate to the query performed.

### Example
In this example, we've built on the previous example to add the `.with_additional` method.

Python
```python
response = (
    client
    .query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_limit(2)
    .with_near_text({"concepts": "Intergalactic travel"})
    .with_additional(["distance", "id"])
    .do()
)
print(json.dumps(response, indent=2))
```

GraphQL
```
{
  Get {
    JeopardyQuestion (
      limit: 2
      nearText: {
        concepts: ["Intergalactic travel"],
      }
    ) {
      question
      answer
      _additional {
        distance
        id
      }
    }
  }
}
```

Again, consider what the response might look like. What would have changed in the response?

Now, try it out yourself. This query should return something like the below:
```
{
  "data": {
    "Get": {
      "JeopardyQuestion": [
        {
          "_additional": {
            "distance": 0.1791926,
            "id": "b1645a32-0c22-5814-8f35-58f142eadf7e"
          },
          "answer": "escaping the Earth's gravity (and go off into outer space, on your way to the moon, for instance)",
          "question": "It takes approximately 24,840 MPH to achieve this"
        },
        {
          "_additional": {
            "distance": 0.18123823,
            "id": "ef263438-b152-5540-97f7-99f4076bd124"
          },
          "answer": "the Milky Way",
          "question": "This is the name of our own galaxy"
        }
      ]
    }
  }
}
```

### Response object from Weaviate
In this response, you see that the `_additional` field has been added to the response object, with `distance` and `id` nested under it.

> Here, the `distance` and `id` properties contain the object's distance to the query and its unique ID respectively.

### `_additonal` properties
As the name suggests, `_additional` properties are separate to those explicitly created as class properties.

The above query included the `distance` property in the `_additional` field. This represents the degree of similarity (or, in this case, the *dissimilarity*) between the input vector and the vector of the object.

> VECTORS AVAILABLE THROUGH `_additional`<br>The object vector can also be retrieved through an `_additional` field, by specifying `vector` in its sub-field.<br>Note that the returned vector in many cases will be a very long list of numbers.

> EXERCISE<br>Try out the above query again, with these changes.<br>- Can you get the `vector` property as well?<br>- Earlier, providing an empty list as the main properties under the `.get()` method will have caused an error. What happens if you try it again, now that you are requesting additional properties?

## Review
### Review exercise
See in-line exercises.

### Key takeaways
- The `Get` function can be used to retrieve objects in Weaviate.
- The `Get` function syntax requires specifying the class, properties, and any additional arguments related to the search criteria.
- Weaviate responses are in JSON format.
- `Class` and `properties` fields must correspond to the objects and properties defined in the Weaviate schema.
- `Get` queries can retrieve additional properties not defined in the schema, which can be inherent to the object or related to the query performed.
- `_additional` properties can include distance, id, and vector information, providing more context and information about the retrieved objects.